In [59]:
from pyspark.sql import SparkSession

In [60]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [61]:
data = spark.read.csv("SMSSpamCollection",inferSchema=True,sep='\t')

In [62]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [63]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [64]:
from pyspark.sql.functions import length

In [65]:
data = data.withColumn('length',length(data['text']))

In [66]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [67]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [68]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer)

In [69]:
tokenizre = Tokenizer(inputCol='text', outputCol='token_text')

In [70]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [71]:
from pyspark.ml.feature import VectorAssembler

In [72]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [73]:
from pyspark.ml.classification import NaiveBayes

In [74]:
nb = NaiveBayes()

In [75]:
from pyspark.ml import Pipeline

In [76]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [77]:
cleaner = data_prep_pipe.fit(data)

In [78]:
clean_data = cleaner.transform(data)

In [79]:
clean_data = clean_data.select('label', 'features')

In [80]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [81]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [82]:
spam_detector = nb.fit(training)

23/09/26 20:50:58 WARN DAGScheduler: Broadcasting large task binary with size 1164.9 KiB
23/09/26 20:50:58 WARN DAGScheduler: Broadcasting large task binary with size 1144.2 KiB


In [83]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [84]:
test_results = spam_detector.transform(test)

In [85]:
test_results.show()

23/09/26 20:50:59 WARN DAGScheduler: Broadcasting large task binary with size 1370.3 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-793.11330785647...|[1.0,8.2058272135...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-884.70441691883...|[1.0,1.8400610840...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1365.0270040539...|[1.0,2.4309241084...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-217.21554004897...|[1.0,7.9024290484...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-696.99451826593...|[1.0,5.8852376777...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-983.43062311610...|[0.99999999989365...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1491.1108907111...|[0.99999981755212...|       0.0|
|  0.0|(13424,[0,1,71,10...|[-687.32803111881...|[1.0,1.7617769985...|       0.0|
|  0.0|(13424,[0,1,150,1...|[-268.06719180995...|[5.26487298279335...|       1.0|
|  0.0|(13424,[0

23/09/26 20:50:59 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [86]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [87]:
acc_eval = MulticlassClassificationEvaluator()

In [88]:
acc = acc_eval.evaluate(test_results)

23/09/26 20:50:59 WARN DAGScheduler: Broadcasting large task binary with size 1374.9 KiB


In [89]:
print('ACC of NB model')
print(acc)

ACC of NB model
0.9142668799204674
